ASAP CRN Metadata validation - wave 1

# ASAP CRN Metadata validation - wave 1

15 September 2023
Andy Henrie



## STEPS

### imports
- pandas
- pathlib

### Load CDE for validation
- check all columns

### Team Lee
- load .tsv, csv tables
- fix format
- load additional metadata

- add batch columns
- add missing columns


### Team Hafler
- load excel file with tables
- add batch info
- add missing columns



### Team Hardy
- load excel file with tables
- add batch info
- add missing columns


In [2]:
import pandas as pd

from pathlib import Path


## Load CDE

In [53]:
CDE_path = Path.cwd() / "CDE.csv" 
CDE = pd.read_csv(CDE_path )

CDE.head()



,Table,Field,Description,DataType,Required,Validation,Unnamed: 6,ClinPath field,team_Hafler type,ClinPath description,Unnamed: 10
0,STUDY,project_name,Project Name: A Title of the overall project...,String,Required,NaN,NaN,NaN,NaN,NaN,bard
1,STUDY,project_dataset,Dataset Name: A unique name is required for ...,String,Required,NaN,NaN,NaN,NaN,NaN,NaN
2,STUDY,project_description,Project Description: Brief description of th...,String,Required,NaN,NaN,NaN,NaN,NaN,NaN
3,STUDY,ASAP_team_name,ASAP Team Name: Name of the ASAP CRN Team. i...,Enum,Required,"[""TEAM-LEE"",""TEAM-HAFLER"",""TEAM-HARDY"", ""TEAM-...",NaN,NaN,NaN,NaN,NaN
4,STUDY,ASAP_lab_name,Lab Name. : Lab name that is submitting data...,String,Required,NaN,NaN,NaN,NaN,NaN,NaN


In [54]:
# # fix trailing ...
# CDE.loc[CDE['Field']=='ASAP_team_name', 'Validation'] = '["TEAM-LEE","TEAM-HAFLER","TEAM-HARDY"]'
# CDE.loc[CDE['Field']=='ASAP_team_name', 'Validation']

In [55]:
# # fix ['Yes","No"]
# CDE.loc[CDE['Field']=='path_infarcs', 'Validation']='["Yes", "No"]'
# CDE.loc[CDE['Field']=='path_infarcs', 'Validation']


In [140]:
def validate_table(table: pd.DataFrame, table_name: str, CDE: pd.DataFrame):
    # Filter out rows specific to the given table_name from the CDE
    specific_cde_df = CDE[CDE['Table'] == table_name]
    
    # Extract fields that have a data type of "Enum" and retrieve their validation entries
    enum_fields_dict = dict(zip(specific_cde_df[specific_cde_df['DataType'] == "Enum"]['Field'], 
                               specific_cde_df[specific_cde_df['DataType'] == "Enum"]['Validation']))
    
    # Extract fields that are marked as "Required"
    required_fields = specific_cde_df[specific_cde_df['Required'] == "Required"]['Field'].tolist()
    optional_fields = specific_cde_df[specific_cde_df['Required'] == "Optional"]['Field'].tolist()

    # Check for missing "Required" fields
    missing_required_fields = [field for field in required_fields if field not in table.columns]
    
    if missing_required_fields:
        print(f"Missing Required Fields in {table_name}: {', '.join(missing_required_fields)}")
    else:
        print(f"All required fields are present in {table_name}.")

    # Check for empty or NaN values in "Required" fields
    empty_or_nan_fields = {}
    for field in required_fields:
        if field in table.columns:
            invalid_count = table[field].isna().sum()
            if invalid_count > 0:
                empty_or_nan_fields[field] = invalid_count
                
    if empty_or_nan_fields:
        print("\nRequired Fields with Empty or NaN values:")
        for field, count in empty_or_nan_fields.items():
            print(f"{field}: {count} rows")
    else:
        print("\nNo empty or NaN values found in required fields.")
    

    # Check for empty or NaN values in "Optional" fields
    empty_or_nan_fields = {}
    for field in optional_fields:
        if field in table.columns:
            invalid_count = table[field].isna().sum()
            if invalid_count > 0:
                empty_or_nan_fields[field] = invalid_count

    if empty_or_nan_fields:
        print("\nOptional fields with Empty or NaN values:")
        for field, count in empty_or_nan_fields.items():
            print(f"{field}: {count} rows")
    else:
        print("\nNo empty or NaN values found in optional fields.")



    # Check for invalid Enum field values
    invalid_field_values = {}
    for field, validation_str in enum_fields_dict.items():
        valid_values = eval(validation_str)
        if field in table.columns:
            invalid_values = table[~table[field].isin(valid_values)][field].unique()
            if invalid_values.any():
                invalid_field_values[field] = invalid_values
    
    if invalid_field_values:
        print("\nInvalid Field/Value pairs:")
        for field, values in invalid_field_values.items():
            print(f"{field}: {', '.join(map(str, values))}")
    else:
        print(f"\nAll Enum fields have valid values in {table_name}.")

    return



## Team Lee

In [81]:
## convert 
data_path = Path.home() / ("Projects/ASAP/team-lee")
metadata_path = data_path / "metadata/ogmetadata"

SUBJECT = pd.read_csv(f"{metadata_path}/SUBJECT.tsv", delimiter="\t")
SAMPLE = pd.read_csv(f"{metadata_path}/SAMPLE.tsv",delimiter="\t")

CLINPATH = pd.read_csv(f"{metadata_path}/CLINPATH.csv",delimiter=",")
STUDY = pd.read_csv(f"{metadata_path}/STUDY.tsv",delimiter="\t")
PROTOCOL = pd.read_csv(f"{metadata_path}/PROTOCOL.tsv",delimiter="\t")


In [83]:
STUDY = pd.read_csv(metadata_path / "STUDY.tsv",delimiter="\t")
STUDY.to_csv(data_path / "STUDY_.csv")
STUDY = pd.read_csv(data_path / "STUDY_.csv")
STUDY.head()


,Unnamed: 0,Unnamed: 1,Unnamed: 2,Team-Lee-Bras-Lab-Info,Field,Description,Data type,Validation,Note,Required/Optional
0,Is senescence a component of human PD and does...,project_name,Project Name/Title,String,NaN,Unique and clear title.,Required,NaN,NaN,NaN
1,Human snRNA-seq PD Senesence Jose Bras Team Lee,project_dataset,Dataset name,String,NaN,A Dataset name is required for each submission...,Required,NaN,NaN,NaN
2,Characterize the neuropathological progression...,project_description,Brief description of the goals and objectives ...,String,NaN,NaN,Required,NaN,NaN,NaN
3,TEAM-LEE,ASAP_team_name,"ASAP Team e.g. ""Scherzer""",Enum,"[""TEAM-LEE"",""TEAM-HAFLER"",""TEAM-HARDY"",....]",NaN,Required,NaN,NaN,NaN
4,Bras,ASAP_lab_name,"ASAP Lab under the above team e.g. ""Dong""",String,NaN,NaN,Required,NaN,NaN,NaN


In [85]:

# fix STUDY formatting
tmp = pd.DataFrame()
tmp = STUDY[["Unnamed: 1","Unnamed: 0"]].transpose().reset_index().drop(columns=["index"])
tmp.columns = tmp.iloc[0]
STUDY = tmp.drop([0])
# STUDY[["Unnamed: 1"]].transpose().reset_index().drop(columns=["index"]), tmp
STUDY.head()

,project_name,project_dataset,project_description,ASAP_team_name,ASAP_lab_name,PI_full_name,PI_email,submitter_id,submitter_name,submittor_email,...,other_funding_source,publication_DOI,publication_PMID,number_of_brain_samples,brain_regions,types_of_samples,PI_ORCHID,PI_google_scholar_id,DUA_version,metadata_version_date
1,Is senescence a component of human PD and does...,Human snRNA-seq PD Senesence Jose Bras Team Lee,Characterize the neuropathological progression...,TEAM-LEE,Bras,"Jose, Bras",jose.bras@vai.org,"Lee, L, Marshall ; Kimberly, E, Paquette ; Kai...",Kaitlyn E Westra,kaitlyn.westra@vai.org,...,NaN,NaN,NaN,75,hippocampus; middle frontal gyrus; substantia ...,human PD and control postmortem brains,NaN,NaN,unsure,NaN


Collect additional metadata from covar.csv .. i.e. batch

In [86]:

metadata_path = Path.home() / ("Projects/ASAP/team-lee/metadata")
HIP_covar = pd.read_csv(f"{metadata_path}/HIP/covar.csv")
HIP_cases = pd.read_csv(f"{metadata_path}/HIP/PD_ASAP_Sample_batch_information_banner_cases.csv").dropna(axis=0,how='all')
HIP_control = pd.read_csv(f"{metadata_path}/HIP/PD_ASAP_Sample_batch_information_banner_controls.csv")

MFG_covar = pd.read_csv(f"{metadata_path}/MFG/covar.csv") # includes 'PMI' ?
MFG_cases = pd.read_csv(f"{metadata_path}/MFG/PD_ASAP_Sample_batch_information_banner_cases.csv").dropna(axis=0,how='all')
MFG_control = pd.read_csv(f"{metadata_path}/MFG/PD_ASAP_Sample_batch_information_banner_controls.csv")


SN_covar = pd.read_csv(f"{metadata_path}/SN/covar.csv")
SN_cases = pd.read_csv(f"{metadata_path}/SN/PD_ASAP_Sample_batch_information_banner_cases.csv").dropna(axis=0,how='all')
SN_control = pd.read_csv(f"{metadata_path}/SN/PD_ASAP_Sample_batch_information_banner_controls.csv")

In [87]:
# Hippocampus samples
# HIP_cases["GROUPcv"]="PD"
# HIP_control["GROUPcv"]="HC"

HIP_meta = pd.concat([HIP_cases, HIP_control], axis=0, ignore_index=True)
HIP_meta["GROUPcv"]= HIP_meta["PD"].apply(lambda x: "PD" if (x=="yes") else "HC")


In [88]:


HIP_meta['MERGE_ID'] = "HIP_" + HIP_meta['GROUPcv'] +"_" + HIP_meta['CaseID'].str.replace('-','')
HIP_covar['MERGE_ID'] = HIP_covar['COUNT_ID']
# the fastqs follow COUNT_ID insteald of SEQ_ID naming convention
HIP_covar['SEQ_ID'] = HIP_covar['COUNT_ID']



In [89]:
# there's a bug in the meta table... skip for now
HIP_TABLE = pd.merge(HIP_covar, HIP_meta, on='MERGE_ID', how='outer')

# HIP_TABLE = HIP_covar
HIP_TABLE['subdir']="HIP"


In [90]:
test = HIP_TABLE[["MERGE_ID","SEQ_ID","GROUPcv","subdir",'PD']]

In [91]:
### medial frontal gyrus samples
MFG_meta = pd.concat([MFG_cases, MFG_control], axis=0, ignore_index=True)
MFG_meta["GROUPcv"]= MFG_meta["PD"].apply(lambda x: "PD" if (x=="yes") else "HC")

# make a MERGE_ID column because the formatting is inconsistent
MFG_meta['MERGE_ID'] = "MFG_" + MFG_meta['GROUPcv'] +"_" + MFG_meta['CaseID'].str.replace('-','')
MFG_covar['MERGE_ID'] = MFG_covar['SAMPLE']
# the fastqs are in SEQ_ID 

# there's a bug in the meta table... skip for now
MFG_TABLE = pd.merge(MFG_covar, MFG_meta, on='MERGE_ID', how='inner')
MFG_TABLE['subdir']="MFG"



# Substantia Nigra
SN_meta = pd.concat([SN_cases, SN_control], axis=0, ignore_index=True)
SN_meta["GROUPcv"] = SN_meta["PD"].apply(lambda x: "PD" if (x=="yes") else "HC")

SN_meta['MERGE_ID'] = "SN_" + MFG_meta['GROUPcv'] +"_" + MFG_meta['CaseID'].str.replace('-','')
SN_covar['MERGE_ID'] = SN_covar['SAMPLE']

# there's a bug in the meta table... skip for now
SN_TABLE = pd.merge(SN_covar, SN_meta, on='MERGE_ID', how='outer')
SN_TABLE['subdir']="SN"


### concatenate SN, MSG, and HIP tables into one 'all_samples' table
all_samples = pd.concat([HIP_TABLE, MFG_TABLE, SN_TABLE], axis=0, ignore_index=True)


In [92]:

SAMPLE_ALL = SAMPLE.merge(all_samples, left_on='sample_id', right_on='MERGE_ID', how='left')
SAMPLE_ALL.to_csv("alternate_metadata.csv")

In [93]:
SAMPLE_og = SAMPLE.copy()
SAMPLE['batch'] = SAMPLE_ALL['BATCH']

In [94]:
# Extract the fields with DataType as "Enum" or "String" for the "sample" table from CDE.csv

# Define a function to only capitalize the first letter of a string
def capitalize_first_letter(s):
    if not isinstance(s, str) or len(s) == 0:  # Check if the value is a string and non-empty
        return s
    return s[0].upper() + s[1:]

def force_enum_string(df, df_name, CDE):

    string_enum_fields = CDE[(CDE["Table"] == df_name) & 
                                (CDE["DataType"].isin(["Enum", "String"]))]["Field"].tolist()
    # Convert the specified columns to string data type using astype() without a loop
    columns_to_convert = {col: 'str' for col in string_enum_fields if col in df.columns}
    df = df.astype(columns_to_convert)

    # enum_fields = CDE[ (CDE["Table"] == df_name) & 
    #                             (CDE["DataType"]=="Enum") ]["Field"].tolist()
    
    for col in string_enum_fields:
        if col in df.columns and col != "assay":
            df[col] = df[col].apply(capitalize_first_letter)

    return df

SAMPLE = force_enum_string(SAMPLE, "SAMPLE", CDE)
# for field in string_enum_fields:
#     if field in SAMPLE.columns:
#         SAMPLE[field] = SAMPLE[field].astype(str)


In [95]:
STUDY

,project_name,project_dataset,project_description,ASAP_team_name,ASAP_lab_name,PI_full_name,PI_email,submitter_id,submitter_name,submittor_email,...,other_funding_source,publication_DOI,publication_PMID,number_of_brain_samples,brain_regions,types_of_samples,PI_ORCHID,PI_google_scholar_id,DUA_version,metadata_version_date
1,Is senescence a component of human PD and does...,Human snRNA-seq PD Senesence Jose Bras Team Lee,Characterize the neuropathological progression...,TEAM-LEE,Bras,"Jose, Bras",jose.bras@vai.org,"Lee, L, Marshall ; Kimberly, E, Paquette ; Kai...",Kaitlyn E Westra,kaitlyn.westra@vai.org,...,NaN,NaN,NaN,75,hippocampus; middle frontal gyrus; substantia ...,human PD and control postmortem brains,NaN,NaN,unsure,NaN


In [96]:
# Testing the function with STUDY.csv and CDE.csv
STUDY.replace("Nan", "", inplace=True)

validate_table(STUDY, "STUDY", CDE)



Missing Required Fields in STUDY: contributor_names

Fields with Empty or NaN values:
other_funding_source: 1 rows
publication_DOI: 1 rows
publication_PMID: 1 rows

Fields with Empty or NaN values:
PI_ORCHID: 1 rows
PI_google_scholar_id: 1 rows
metadata_version_date: 1 rows

All Enum fields have valid values in STUDY.


In [97]:

# Need to rename submitter_id to contributor_names
STUDY = STUDY.rename(columns={"submitter_id": "contributor_names"})
validate_table(STUDY, "STUDY", CDE)


All required fields are present in STUDY.

Fields with Empty or NaN values:
other_funding_source: 1 rows
publication_DOI: 1 rows
publication_PMID: 1 rows

Fields with Empty or NaN values:
PI_ORCHID: 1 rows
PI_google_scholar_id: 1 rows
metadata_version_date: 1 rows

All Enum fields have valid values in STUDY.


In [98]:
# Testing the function with PROTOCOL.csv and CDE.csv

validate_table(PROTOCOL, "PROTOCOL", CDE)

All required fields are present in PROTOCOL.

No empty or NaN values found in required fields.

Fields with Empty or NaN values:
other_reference: 1 rows

All Enum fields have valid values in PROTOCOL.


In [99]:
# Extract the fields with DataType as "Enum" or "String" for the "sample" table from CDE.csv
SUBJECT.replace("Nan", "", inplace=True)


SUBJECT = force_enum_string(SUBJECT, "SUBJECT", CDE)

# Testing the function with SUBJECT.csv and CDE.csv
validate_table(SUBJECT, "SUBJECT", CDE)

All required fields are present in SUBJECT.

No empty or NaN values found in required fields.

Fields with Empty or NaN values:
primary_diagnosis_text: 23 rows

All Enum fields have valid values in SUBJECT.


In [100]:
SAMPLE.replace("Nan", "", inplace=True)

SAMPLE = force_enum_string(SAMPLE, "SAMPLE", CDE)


# Testing the function with SAMPLE.csv and CDE.csv
validate_table(SAMPLE, "SAMPLE", CDE)

# sequence length will need to be converted to a string




Missing Required Fields in SAMPLE: file_MD5

Fields with Empty or NaN values:
source_RIN: 75 rows
RIN: 75 rows

Fields with Empty or NaN values:
pm_PH: 75 rows

Invalid Field/Value pairs:
file_type: Fastq


In [101]:
SAMPLE['file_type'] = SAMPLE['file_type'].replace({"Fastq":"fastq"})


In [102]:
# make the colunn order of SAMPLE match the CDE.Field
# SAMPLE = SAMPLE[CDE.Field.tolist()]
SAMPLE.head()

,sample_id,source_sample_id,subject_id,replicate,replicate_count,repeated_sample,tissue,brain_region,source_RIN,RIN,...,self_reported_ethnicity_ontology_term_id,disease_ontology_term_id,tissue_ontology_term_id,cell_type_ontology_term_id,assay_ontology_term_id,suspension_type,DV2000,pm_PH,donor_id,batch
0,MFG_HC_1225,12-25,12-25,Rep1,1,0,Brain,Middle_Frontal_Gyrus,NaN,NaN,...,Unknown,PATO:0000461,UBERON:0002702,NA(multiple),EFO:0030004,nucleus,NaN,NaN,,BATCH_4
1,MFG_HC_0602,06-02,06-02,Rep1,1,0,Brain,Middle_Frontal_Gyrus,NaN,NaN,...,Unknown,PATO:0000461,UBERON:0002702,NA(multiple),EFO:0030004,nucleus,NaN,NaN,,BATCH_4
2,MFG_PD_0009,00-09,00-09,Rep1,1,0,Brain,Middle_Frontal_Gyrus,NaN,NaN,...,Unknown,MONDO:0005180,UBERON:0002702,NA(multiple),EFO:0030004,nucleus,NaN,NaN,,BATCH_4
3,MFG_PD_1921,19-21,19-21,Rep1,1,0,Brain,Middle_Frontal_Gyrus,NaN,NaN,...,Unknown,MONDO:0005180,UBERON:0002702,NA(multiple),EFO:0030004,nucleus,NaN,NaN,,BATCH_4
4,MFG_PD_2058,20-58,20-58,Rep1,1,0,Brain,Middle_Frontal_Gyrus,NaN,NaN,...,Unknown,MONDO:0005180,UBERON:0002702,NA(multiple),EFO:0030004,nucleus,NaN,NaN,,BATCH_4


In [103]:
# fix file_name and file_MD5 which need to be exploded (do this last for simplicity. i.e. to keep one sample per row rather than one file per row)

# Step 1: Split the values in the columns based on commas
SAMPLE['file_name'] = SAMPLE['file_name'].str.split(',')
SAMPLE['file_MD5(R1,R2)'] = SAMPLE['file_MD5(R1,R2)'].str.split(',')

# Step 2: Explode both 'file_name' and 'file_MD5(R1,R2)' columns together
SAMPLE = SAMPLE.explode(['file_name', 'file_MD5(R1,R2)'])

# Step 3: Rename the "file_MD5(R1,R2)" column to "file_MD5"
SAMPLE = SAMPLE.rename(columns={"file_MD5(R1,R2)": "file_MD5"})



In [107]:
SAMPLE.head()

,sample_id,source_sample_id,subject_id,replicate,replicate_count,repeated_sample,tissue,brain_region,source_RIN,RIN,...,self_reported_ethnicity_ontology_term_id,disease_ontology_term_id,tissue_ontology_term_id,cell_type_ontology_term_id,assay_ontology_term_id,suspension_type,DV2000,pm_PH,donor_id,batch
0,MFG_HC_1225,12-25,12-25,Rep1,1,0,Brain,Middle_Frontal_Gyrus,NaN,NaN,...,Unknown,PATO:0000461,UBERON:0002702,NA(multiple),EFO:0030004,nucleus,NaN,NaN,,BATCH_4
0,MFG_HC_1225,12-25,12-25,Rep1,1,0,Brain,Middle_Frontal_Gyrus,NaN,NaN,...,Unknown,PATO:0000461,UBERON:0002702,NA(multiple),EFO:0030004,nucleus,NaN,NaN,,BATCH_4
1,MFG_HC_0602,06-02,06-02,Rep1,1,0,Brain,Middle_Frontal_Gyrus,NaN,NaN,...,Unknown,PATO:0000461,UBERON:0002702,NA(multiple),EFO:0030004,nucleus,NaN,NaN,,BATCH_4
1,MFG_HC_0602,06-02,06-02,Rep1,1,0,Brain,Middle_Frontal_Gyrus,NaN,NaN,...,Unknown,PATO:0000461,UBERON:0002702,NA(multiple),EFO:0030004,nucleus,NaN,NaN,,BATCH_4
2,MFG_PD_0009,00-09,00-09,Rep1,1,0,Brain,Middle_Frontal_Gyrus,NaN,NaN,...,Unknown,MONDO:0005180,UBERON:0002702,NA(multiple),EFO:0030004,nucleus,NaN,NaN,,BATCH_4


In [108]:
CLINPATH = force_enum_string(CLINPATH, "CLINPATH", CDE)
CLINPATH.replace("Nan", "", inplace=True)

# Testing the function with CLINPATH.csv and CDE.csv
validate_table(CLINPATH, "CLINPATH", CDE)

All required fields are present in CLINPATH.

Fields with Empty or NaN values:
age_at_onset: 75 rows
age_at_diagnosis: 75 rows
first_motor_symptom: 75 rows
path_year_death: 75 rows
brain_weight: 75 rows

Fields with Empty or NaN values:
smoking_years: 75 rows

Invalid Field/Value pairs:
region_level_2: Hippocampus
path_braak_asyn: L. Olfactory Bulb-Only, Lla. Brainstem Predominant, Llb. Limbic Predominant, LV. Neocortical, Lll. Brainstem/Limbic, 0. No Lewy bodies
path_mckeith: L. Olfactory Bulb-Only, Lla. Brainstem Predominant, Llb. Limbic Predominant, LV. Neocortical, Lll. Brainstem/Limbic, 0. No Lewy bodies
path_nia_ri: Criteria not met, Not AD
TDP43: , Na
amyloid_angiopathy_severity_scale: , Cerebral amyloid angiopathy, temporal and occipital lobe, Cerebral amyloid angiopathy, frontal lobe
path_ad_level: Microscopic changes of Alzheimer's disease, insufficient for diagnosis, Microscopic lesions of Alzheimer's disease, insufficient for diagnosis


In [109]:
CLINPATH['region_level_2'].unique()

# change "Hippocampus" to "CA1-CA4"
CLINPATH['region_level_2'] = CLINPATH['region_level_2'].replace('Hippocampus', 'CA1-CA4')

# skip hx_melanoma and education level for now as there is not a "Unknown" or "Not Reported" option in the CDE

# leave te APOE_e4_status as is for now . multiple are coded as "2,3" 
# leave cognitive status as is, since there is no "Unknown" or "Not Reported" option in the CDE

# potential "path_braak_asyn" coding 
braak_map = {'L. Olfactory Bulb-Only':"1/2", 'Lla. Brainstem Predominant':"3",
       'Llb. Limbic Predominant':"3/4", 'LV. Neocortical':"5",
       'Lll. Brainstem/Limbic':"3/4", '0. No Lewy bodies':"0"}
# set to NaN for now since this is actualy path_mckeith coding

CLINPATH['path_braak_asyn'] = ""

mckeith_map = {'L. Olfactory Bulb-Only':"Olfactory bulb only", 'Lla. Brainstem Predominant':"Brainstem",
       'Llb. Limbic Predominant':"Limbic (transitional)", 'LV. Neocortical':"Neocortical",
       'Lll. Brainstem/Limbic':"Amygdala Predominant", '0. No Lewy bodies':"Absent"}


CLINPATH['path_mckeith'] = CLINPATH['path_mckeith'].replace(mckeith_map)

# leave path_nia_ri like this for now. not sure how to map "criteria not met" and "Not AD"

# leave amyloid_angiopathy_severity_scale like this for now. not sure how to map 'Cerebral amyloid angiopathy, temporal and occipital lobe','Cerebral amyloid angiopathy, frontal lobe']


In [111]:
validate_table(CLINPATH, "CLINPATH", CDE)

All required fields are present in CLINPATH.

Fields with Empty or NaN values:
age_at_onset: 75 rows
age_at_diagnosis: 75 rows
first_motor_symptom: 75 rows
path_year_death: 75 rows
brain_weight: 75 rows

Fields with Empty or NaN values:
smoking_years: 75 rows

Invalid Field/Value pairs:
path_nia_ri: Criteria not met, Not AD
TDP43: , Na
amyloid_angiopathy_severity_scale: , Cerebral amyloid angiopathy, temporal and occipital lobe, Cerebral amyloid angiopathy, frontal lobe
path_ad_level: Microscopic changes of Alzheimer's disease, insufficient for diagnosis, Microscopic lesions of Alzheimer's disease, insufficient for diagnosis


In [112]:
CLINPATH

,sample_id,source_sample_id,time_from_baseline,GP2_id,hemisphere,region_level_1,region_level_2,region_level_3,AMPPD_id,family_history,...,sn_neuronal_loss,path_infarcs,path_nia_ri,path_nia_aa_a,path_nia_aa_b,path_nia_aa_c,TDP43,arteriolosclerosis_severity_scale,amyloid_angiopathy_severity_scale,path_ad_level
0,MFG_HC_1225,12-25,0,,Unknown,Frontal lobe,Middle frontal gyrus,Unknown,,Not Reported,...,,,Criteria not met,,,,,,,No evidence of Alzheimer's disease neuropathol...
1,MFG_HC_0602,06-02,0,,Unknown,Frontal lobe,Middle frontal gyrus,Unknown,,Not Reported,...,,,Criteria not met,,,,,,,No evidence of Alzheimer's disease neuropathol...
2,MFG_PD_0009,00-09,0,,Unknown,Frontal lobe,Middle frontal gyrus,Unknown,,Not Reported,...,,,Not AD,,,,,,,No evidence of Alzheimer's disease neuropathol...
3,MFG_PD_1921,19-21,0,,Unknown,Frontal lobe,Middle frontal gyrus,Unknown,,Not Reported,...,,,Criteria not met,,,,None in medial temporal lobe,,,No evidence of Alzheimer's disease neuropathol...
4,MFG_PD_2058,20-58,0,,Unknown,Frontal lobe,Middle frontal gyrus,Unknown,,Not Reported,...,,,Low,,,,None in medial temporal lobe,,,"Microscopic changes of Alzheimer's disease, in..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,SN_PD_1858,18-58,0,,Unknown,Midbrain,Substantia nigra,Unknown,,Not Reported,...,,,Criteria not met,,,,Unknown,,,"Microscopic changes of Alzheimer's disease, in..."
71,SN_PD_1902,19-02,0,,Unknown,Midbrain,Substantia nigra,Unknown,,Not Reported,...,,,Criteria not met,,,,None in medial temporal lobe,,,No evidence of Alzheimer's disease neuropathol...
72,SN_PD_1973,19-73,0,,Unknown,Midbrain,Substantia nigra,Unknown,,Not Reported,...,,Yes,Criteria not met,,,,None in medial temporal lobe,,,No evidence of Alzheimer's disease neuropathol...
73,SN_PD_2005,20-05,0,,Unknown,Midbrain,Substantia nigra,Unknown,,Not Reported,...,,Yes,Criteria not met,,,,None in medial temporal lobe,,,No evidence of Alzheimer's disease neuropathol...


In [110]:

SAMPLE_ALL_CP = SAMPLE_ALL.merge(CLINPATH, on='sample_id', how='outer')


In [131]:
SAMPLE_ALL_CP.to_csv("./clean/team-Lee/auxiluary_metadata.csv")

## write clean metadata tables according to CDE 

In [113]:
# SAMPLE = SAMPLE[CDE[CDE["Table"]=="SAMPLE"].Field.tolist()]


def reorder_table_to_CDE(df, df_name, CDE):
    col_order = CDE[CDE["Table"]==df_name].Field.tolist()
    
    df_out = pd.DataFrame()
    for col in col_order:
        if col in df.columns:   
            df_out[col] = df[col]
        else:
            df_out[col] = ""

    return df_out



In [114]:
# fix the column order
STUDY = reorder_table_to_CDE(STUDY, "STUDY", CDE)
SAMPLE = reorder_table_to_CDE(SAMPLE, "SAMPLE", CDE)
PROTOCOL = reorder_table_to_CDE(PROTOCOL, "PROTOCOL", CDE)
SUBJECT = reorder_table_to_CDE(SUBJECT, "SUBJECT", CDE)     
CLINPATH = reorder_table_to_CDE(CLINPATH, "CLINPATH", CDE)

# write the clean metadata
STUDY.to_csv(data_path / "metadata/STUDY.csv")
PROTOCOL.to_csv(data_path / "metadata/PROTOCOL.csv")
CLINPATH.to_csv(data_path / "metadata/CLINPATH.csv")
SAMPLE.to_csv(data_path / "metadata/SAMPLE.csv")
SUBJECT.to_csv(data_path / "metadata/SUBJECT.csv")

# also writh them to clean...
# 
#  

export_root = Path.cwd() / "clean/team-Lee"
if not export_root.exists():
    export_root.mkdir(parents=True, exist_ok=True)


STUDY.to_csv( export_root / "STUDY.csv")
PROTOCOL.to_csv(export_root / "PROTOCOL.csv")
SAMPLE.to_csv(export_root / "SAMPLE.csv")
SUBJECT.to_csv(export_root / "SUBJECT.csv")
CLINPATH.to_csv(export_root / "CLINPATH.csv")


In [115]:
CLINPATH

,sample_id,source_sample_id,time_from_baseline,GP2_id,hemisphere,region_level_1,region_level_2,region_level_3,AMPPD_id,family_history,...,path_nia_ri,path_nia_aa_a,path_nia_aa_b,path_nia_aa_c,TDP43,arteriolosclerosis_severity_scale,amyloid_angiopathy_severity_scale,path_ad_level,dig_slide_avail,quant_path_avail
0,MFG_HC_1225,12-25,0,,Unknown,Frontal lobe,Middle frontal gyrus,Unknown,,Not Reported,...,Criteria not met,,,,,,,No evidence of Alzheimer's disease neuropathol...,,
1,MFG_HC_0602,06-02,0,,Unknown,Frontal lobe,Middle frontal gyrus,Unknown,,Not Reported,...,Criteria not met,,,,,,,No evidence of Alzheimer's disease neuropathol...,,
2,MFG_PD_0009,00-09,0,,Unknown,Frontal lobe,Middle frontal gyrus,Unknown,,Not Reported,...,Not AD,,,,,,,No evidence of Alzheimer's disease neuropathol...,,
3,MFG_PD_1921,19-21,0,,Unknown,Frontal lobe,Middle frontal gyrus,Unknown,,Not Reported,...,Criteria not met,,,,None in medial temporal lobe,,,No evidence of Alzheimer's disease neuropathol...,,
4,MFG_PD_2058,20-58,0,,Unknown,Frontal lobe,Middle frontal gyrus,Unknown,,Not Reported,...,Low,,,,None in medial temporal lobe,,,"Microscopic changes of Alzheimer's disease, in...",,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,SN_PD_1858,18-58,0,,Unknown,Midbrain,Substantia nigra,Unknown,,Not Reported,...,Criteria not met,,,,Unknown,,,"Microscopic changes of Alzheimer's disease, in...",,
71,SN_PD_1902,19-02,0,,Unknown,Midbrain,Substantia nigra,Unknown,,Not Reported,...,Criteria not met,,,,None in medial temporal lobe,,,No evidence of Alzheimer's disease neuropathol...,,
72,SN_PD_1973,19-73,0,,Unknown,Midbrain,Substantia nigra,Unknown,,Not Reported,...,Criteria not met,,,,None in medial temporal lobe,,,No evidence of Alzheimer's disease neuropathol...,,
73,SN_PD_2005,20-05,0,,Unknown,Midbrain,Substantia nigra,Unknown,,Not Reported,...,Criteria not met,,,,None in medial temporal lobe,,,No evidence of Alzheimer's disease neuropathol...,,


## Team Hafler

In [116]:
## convert to seurat Object
data_path = Path.home() / ("Projects/ASAP")
metadata_path = data_path / "team-hafler/metadata"

sheets = ["SAMPLE","SUBJECT","CLINPATH","STUDY","PROTOCOL"]
excel_path = data_path / "ASAP_CDE_ALL_Team_Hafler_v1.xlsx"
STUDY = pd.read_excel(excel_path,sheet_name="STUDY",header=1).drop(columns="Field")
CLINPATH = pd.read_excel(excel_path,sheet_name="CLINPATH",header=1).drop(columns="Field")
SUBJECT = pd.read_excel(excel_path,sheet_name="SUBJECT",header=1).drop(columns="Field")
SAMPLE = pd.read_excel(excel_path,sheet_name="SAMPLE",header=1).drop(columns="Field")
PROTOCOL = pd.read_excel(excel_path,sheet_name="PROTOCOL",header=1).drop(columns="Field")
metadata_path

PosixPath('/Users/ergonyc/Projects/ASAP/team-hafler/metadata')

In [117]:

def add_hafler_batch(sample_df):

    # First batch: HSDG07HC HSDG10HC HSDG148PD HSDG199PD
    # batch[batch.sample_id in ['hSDG07HC', 'hSDG10HC', 'hSDG148PD', 'hSDG199PD']]=1
    Batch_1 = ['hSDG07', 'hSDG10', 'hSDG148', 'hSDG199'] 
    # Second batch: hsDG101HC hsDG13HC hsDG151PD hsDG197PD hsDG30HC hsDG99HC
    Batch_2 = ['hSDG101', 'hSDG13', 'hSDG151', 'hSDG197', 'hSDG30', 'hSDG99']
    # Third batch: hsDG142PD hsDG208PD
    Batch_3 = ['hSDG142', 'hSDG208'] 


    batch_col = []
    for row in sample_df.sample_id:
        if row in Batch_1:
            batch_col.append("Batch_1")
        elif row in Batch_2:
            batch_col.append("Batch_2")
        elif row in Batch_3:
            batch_col.append("Batch_3")
        else:
            print("ERROR >>>>>>>> not no batch info")
            batch_col.append("")


    sample_df['batch'] = batch_col
    return sample_df

SAMPLE = add_hafler_batch(SAMPLE)

In [118]:
# fix replicate & replicate_count
SAMPLE.replace("Nan", "", inplace=True)

SAMPLE['replicate'] = "Rep1"
SAMPLE['replicate_count'] = 1

SAMPLE.head(50)

,sample_id,source_sample_id,subject_id,replicate,replicate_count,repeated_sample,batch,tissue,brain_region,source_RIN,...,sex_ontology_term_id,self_reported_ethnicity_ontology_term_id,disease_ontology_term_id,tissue_ontology_term_id,cell_type_ontology_term_id,assay_ontology_term_id,suspension_type,DV2000,pm_PH,donor_id
0,hSDG07,hSDG07,HC01,Rep1,1,0,Batch_1,Brain,Prefrontal Cortex,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,hSDG07,hSDG07,HC01,Rep1,1,0,Batch_1,Brain,Prefrontal Cortex,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,hSDG07,hSDG07,HC01,Rep1,1,0,Batch_1,Brain,Prefrontal Cortex,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,hSDG101,hSDG101,HC03,Rep1,1,0,Batch_2,Brain,Prefrontal Cortex,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,hSDG101,hSDG101,HC03,Rep1,1,0,Batch_2,Brain,Prefrontal Cortex,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,hSDG101,hSDG101,HC03,Rep1,1,0,Batch_2,Brain,Prefrontal Cortex,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,hSDG10,hSDG10,HC04,Rep1,1,0,Batch_1,Brain,Prefrontal Cortex,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,hSDG10,hSDG10,HC04,Rep1,1,0,Batch_1,Brain,Prefrontal Cortex,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,hSDG10,hSDG10,HC04,Rep1,1,0,Batch_1,Brain,Prefrontal Cortex,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,hSDG13,hSDG13,HC02,Rep1,1,0,Batch_2,Brain,Prefrontal Cortex,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [119]:

STUDY = force_enum_string(STUDY, "STUDY", CDE)


In [120]:
# Testing the function with STUDY.csv and CDE.csv
validate_table(STUDY, "STUDY", CDE)


# Need to add contributor_names

All required fields are present in STUDY.

No empty or NaN values found in required fields.

No empty or NaN values found in optional fields.

All Enum fields have valid values in STUDY.


In [121]:
# Testing the function with PROTOCOL.csv and CDE.csv
validate_table(PROTOCOL, "PROTOCOL", CDE)

All required fields are present in PROTOCOL.

Fields with Empty or NaN values:
github_url: 1 rows
protocols_io_DOI: 1 rows

Fields with Empty or NaN values:
other_reference: 1 rows

All Enum fields have valid values in PROTOCOL.


In [122]:
SUBJECT.replace("Nan", "", inplace=True)
# Testing the function with SUBJECT.csv and CDE.csv
validate_table(SUBJECT, "SUBJECT", CDE)

All required fields are present in SUBJECT.

No empty or NaN values found in required fields.

Fields with Empty or NaN values:
primary_diagnosis_text: 12 rows

Invalid Field/Value pairs:
sex: F, M
race: B, W
primary_diagnosis: normal control, idiopathic Parkinson's disease


In [123]:

SUBJECT = force_enum_string(SUBJECT, "SUBJECT", CDE)


In [125]:
SUBJECT['sex'] = SUBJECT['sex'].replace({'F':"Female", 'M':"Male"})
SUBJECT['race'] = SUBJECT['race'].replace({'W':"White", 'B':"Black or African American"})

SUBJECT['primary_diagnosis'] = SUBJECT['primary_diagnosis'].replace({'Normal control':"Healthy Control", "Idiopathic Parkinson's disease":"Idiopathic PD"})
validate_table(SUBJECT, "SUBJECT", CDE)

All required fields are present in SUBJECT.

No empty or NaN values found in required fields.

Fields with Empty or NaN values:
primary_diagnosis_text: 12 rows

All Enum fields have valid values in SUBJECT.


In [133]:
SAMPLE.replace("Nan", "", inplace=True)
SAMPLE.replace("nan", "", inplace=True)

# Testing the function with SAMPLE.csv and CDE.csv
validate_table(SAMPLE, "SAMPLE", CDE)

# sequence length will need to be converted to a string

All required fields are present in SAMPLE.

Fields with Empty or NaN values:
source_RIN: 36 rows
RIN: 36 rows
suspension_type: 36 rows

Fields with Empty or NaN values:
pm_PH: 36 rows

All Enum fields have valid values in SAMPLE.


In [127]:
SAMPLE = force_enum_string(SAMPLE, "SAMPLE", CDE)


In [128]:
# force the right sex_ontology_term_id
SAMPLE["organism_ontology_term_id"] = "NCBITaxon:9606"

# set time == 0 for all samples
SAMPLE['time'] = 0

SAMPLE['file_type'] = SAMPLE['file_type'].replace({"Fastq":"fastq"})


In [129]:

# need to join with subject to get "sex" and convert to ontology term
SAMPLE_SUBJECT = SAMPLE.merge(SUBJECT, on='subject_id',  how='left')
SAMPLE_og = SAMPLE.copy()
SAMPLE['sex_ontology_term_id'] = SAMPLE_SUBJECT['sex'].replace({"Male":"PATO:0000384 (male)", "Female":"PATO:0000383 (female)" })

# ignore development_stage_ontology_term_id, self_reported_ethnicity_ontology_term_id, assay_ontology_term_id, etc for now. (Check wiht Le)

In [130]:
# fix assay
SAMPLE['assay'] = SAMPLE['assay'].replace({'v3.1 - Single Index, 10x Genomics ':"v3.1 - Single Index"})
# fix assay
SAMPLE['sequencing_length'] = SAMPLE['sequencing_length'].replace({'150bp x2':"150"})


In [134]:
validate_table(SAMPLE, "SAMPLE", CDE)

All required fields are present in SAMPLE.

Fields with Empty or NaN values:
source_RIN: 36 rows
RIN: 36 rows
suspension_type: 36 rows

Fields with Empty or NaN values:
pm_PH: 36 rows

All Enum fields have valid values in SAMPLE.


In [135]:
CLINPATH.replace("Nan", "", inplace=True)
CLINPATH.replace("nan", "", inplace=True)

# Testing the function with CLINPATH.csv and CDE.csv
validate_table(CLINPATH, "CLINPATH", CDE)

Missing Required Fields in CLINPATH: path_thal

Fields with Empty or NaN values:
time_from_baseline: 12 rows
GP2_id: 12 rows
AMPPD_id: 12 rows
last_diagnosis: 12 rows
age_at_onset: 12 rows
age_at_diagnosis: 12 rows
first_motor_symptom: 12 rows
hx_dementia_mci: 12 rows
hx_melanoma: 12 rows
education_level: 12 rows
smoking_status: 12 rows
path_autopsy_dx_main: 6 rows
path_year_death: 12 rows
cause_death: 12 rows
brain_weight: 12 rows
path_braak_nft: 12 rows
path_braak_asyn: 12 rows
path_cerad: 12 rows

Fields with Empty or NaN values:
smoking_years: 12 rows
path_autopsy_second_dx: 12 rows
path_autopsy_third_dx: 12 rows
path_autopsy_fourth_dx: 12 rows
path_autopsy_fifth_dx: 12 rows
path_autopsy_sixth_dx: 12 rows
path_autopsy_seventh_dx: 12 rows
path_autopsy_eight_dx: 12 rows
other_cause_death_1: 12 rows
other_cause_death_2: 12 rows
known_pathogenic_mutation: 12 rows
PD_pathogenic_mutation: 12 rows
path_mckeith: 12 rows
sn_neuronal_loss: 12 rows
path_infarcs: 12 rows
path_nia_ri: 12 rows
p

In [136]:
CLINPATH = force_enum_string(CLINPATH, "CLINPATH", CDE)


In [137]:
# redact "Prefrontal Cortex" from region_level_2 for now
CLINPATH['region_level_2'] = CLINPATH['region_level_2'].replace({'Prefrontal Cortex':""})

# leave te APOE_e4_status as is for now . multiple are coded as "2,3" 
# but remove commas
CLINPATH["APOE_e4_status"] = CLINPATH["APOE_e4_status"].str.replace(",","")

# need to fix the path_autopsy_dx_main

In [139]:
validate_table(CLINPATH, "CLINPATH", CDE)
CLINPATH.head()

Missing Required Fields in CLINPATH: path_thal

Fields with Empty or NaN values:
age_at_onset: 12 rows
age_at_diagnosis: 12 rows
first_motor_symptom: 12 rows
path_year_death: 12 rows
brain_weight: 12 rows

Fields with Empty or NaN values:
smoking_years: 12 rows

Invalid Field/Value pairs:
region_level_2: Prefrontal cortex
hx_dementia_mci: Nan
hx_melanoma: Nan
education_level: Nan
smoking_status: Nan
path_autopsy_dx_main: Nan
path_braak_nft: Nan
path_braak_asyn: Nan
path_cerad: Nan
known_pathogenic_mutation: Nan
path_mckeith: Nan
sn_neuronal_loss: Nan
path_infarcs: Nan
path_nia_ri: Nan
path_nia_aa_a: Nan
path_nia_aa_b: Nan
path_nia_aa_c: Nan
TDP43: Nan
arteriolosclerosis_severity_scale: Nan
amyloid_angiopathy_severity_scale: Nan
path_ad_level: Nan


,sample_id,source_sample_id,time_from_baseline,GP2_id,hemisphere,region_level_1,region_level_2,region_level_3,AMPPD_id,family_history,...,sn_neuronal_loss,path_infarcs,path_nia_ri,path_nia_aa_a,path_nia_aa_b,path_nia_aa_c,TDP43,arteriolosclerosis_severity_scale,amyloid_angiopathy_severity_scale,path_ad_level
0,HC01,HSDG07,Nan,Nan,Unknown,Frontal lobe,Prefrontal cortex,Grey matter,Nan,Unknown,...,Nan,Nan,Nan,Nan,Nan,Nan,Nan,Nan,Nan,Nan
1,HC02,HSDG13,Nan,Nan,Unknown,Frontal lobe,Prefrontal cortex,Grey matter,Nan,Unknown,...,Nan,Nan,Nan,Nan,Nan,Nan,Nan,Nan,Nan,Nan
2,HC03,HSDG101,Nan,Nan,Unknown,Frontal lobe,Prefrontal cortex,Grey matter,Nan,Unknown,...,Nan,Nan,Nan,Nan,Nan,Nan,Nan,Nan,Nan,Nan
3,HC04,HSDG10,Nan,Nan,Unknown,Frontal lobe,Prefrontal cortex,Grey matter,Nan,Unknown,...,Nan,Nan,Nan,Nan,Nan,Nan,Nan,Nan,Nan,Nan
4,HC05,HSDG30,Nan,Nan,Unknown,Frontal lobe,Prefrontal cortex,Grey matter,Nan,Unknown,...,Nan,Nan,Nan,Nan,Nan,Nan,Nan,Nan,Nan,Nan


In [141]:
data_path = data_path / "team-hafler"
data_path = Path.home() / ("Projects/ASAP/team-hafler")

In [142]:
# fix the column order
STUDY = reorder_table_to_CDE(STUDY, "STUDY", CDE)
SAMPLE = reorder_table_to_CDE(SAMPLE, "SAMPLE", CDE)
PROTOCOL = reorder_table_to_CDE(PROTOCOL, "PROTOCOL", CDE)
SUBJECT = reorder_table_to_CDE(SUBJECT, "SUBJECT", CDE)     
CLINPATH = reorder_table_to_CDE(CLINPATH, "CLINPATH", CDE)

# write the clean metadata
STUDY.to_csv(data_path / "metadata/STUDY.csv")
PROTOCOL.to_csv(data_path / "metadata/PROTOCOL.csv")
CLINPATH.to_csv(data_path / "metadata/CLINPATH.csv")
SAMPLE.to_csv(data_path / "metadata/SAMPLE.csv")
SUBJECT.to_csv(data_path / "metadata/SUBJECT.csv")

# also writh them to clean...
# 
#  

export_root = Path.cwd() / "clean/team-Hafler"
if not export_root.exists():
    export_root.mkdir(parents=True, exist_ok=True)


STUDY.to_csv( export_root / "STUDY.csv")
PROTOCOL.to_csv(export_root / "PROTOCOL.csv")
SAMPLE.to_csv(export_root / "SAMPLE.csv")
SUBJECT.to_csv(export_root / "SUBJECT.csv")
CLINPATH.to_csv(export_root / "CLINPATH.csv")


In [143]:
STUDY.to_csv( metadata_path / "STUDY.csv")
CLINPATH.to_csv( metadata_path / "CLINPATH.csv")
SUBJECT.to_csv( metadata_path / "SUBJECT.csv")
SAMPLE.to_csv( metadata_path / "SAMPLE.csv")
PROTOCOL.to_csv( metadata_path / "PROTOCOL.csv")

In [144]:
CLINPATH["APOE_e4_status"].unique()

CDE[CDE["Field"]== "APOE_e4_status"].Validation.unique()

array(['["22", "23","24","33", "34", "44", "Unknown"]'], dtype=object)

## Team Hardy

In [146]:
## convert 
data_path = Path.home() / ("Projects/ASAP/team-hardy")
metadata_path = data_path / "metadata"

SUBJECT = pd.read_csv(f"{metadata_path}/SUBJECT.csv")
CLINPATH = pd.read_csv(f"{metadata_path}/CLINPATH.csv")
STUDY = pd.read_csv(f"{metadata_path}/STUDY.csv")
PROTOCOL = pd.read_csv(f"{metadata_path}/PROTOCOL.csv")
SAMPLE = pd.read_csv(f"{metadata_path}/SAMPLE.csv")


In [154]:
STUDY.replace("NA", "", inplace=True)

validate_table(STUDY, "STUDY", CDE)
STUDY

Missing Required Fields in STUDY: project_name, project_dataset, project_description, ASAP_team_name, ASAP_lab_name, PI_full_name, PI_email, contributor_names, submitter_name, submittor_email, ASAP_grant_id, other_funding_source, publication_DOI, publication_PMID, number_of_brain_samples, brain_regions, types_of_samples, DUA_version

No empty or NaN values found in required fields.

No empty or NaN values found in optional fields.

All Enum fields have valid values in STUDY.


,name,value
0,project_name,Understanding mechanisms of Parkinson's diseas...
1,project_dataset,Hardy snRNA-seq
2,project_description,Genetic analysis has identified many risk gene...
3,ASAP_team_name,TEAM-HARDY
4,ASAP_lab_name,Ryten Lab
5,PI_full_name,Mina Ryten
6,PI_email,mina.ryten@ucl.ac.uk
7,contributor_names,"Aine Fairbrother-Browne, Jonathan Brenton, Mel..."
8,submitter_name,Aine Fairbrother-Browne
9,submitter_email,aine.fairbrother-browne.18@ucl.ac.uk


In [151]:
# there seems to be something funky with SAMPLE
# SAMPLE = SAMPLE[SAMPLE["batch"]=="B1"]
# SAMPLE.drop_duplicates(inplace=True) #, subset=[ "file_name"])
SAMPLE

,sample_id,source_subject_id,subject_id,replicate,replicate_count,repeated_sample,batch,tissue,brain_region,source_RIN,...,sex_ontology_term_id,self_reported_ethnicity_ontology_term_id,disease_ontology_term_id,tissue_ontology_term_id,cell_type_ontology_term_id,assay_ontology_term_id,suspension_type,DV2000,pm_PH,donor_id
0,babom_ACG,P2/14,babom,NaN,1,0,B1,brain,ACG,NaN,...,PATO:0000383 (female),NaN,MONDO:0005180,UBERON:0009835,NaN,EFO:0008913,nucleus,NaN,NaN,NaN
1,babom_ACG,P2/14,babom,NaN,1,0,B1,brain,ACG,NaN,...,PATO:0000383 (female),NaN,MONDO:0005180,UBERON:0009835,NaN,EFO:0008913,nucleus,NaN,NaN,NaN
2,babom_ACG,P2/14,babom,NaN,1,0,B1,brain,ACG,NaN,...,PATO:0000383 (female),NaN,MONDO:0005180,UBERON:0009835,NaN,EFO:0008913,nucleus,NaN,NaN,NaN
3,babom_ACG,P2/14,babom,NaN,1,0,B1,brain,ACG,NaN,...,PATO:0000383 (female),NaN,MONDO:0005180,UBERON:0009835,NaN,EFO:0008913,nucleus,NaN,NaN,NaN
4,babom_ACG,P2/14,babom,NaN,1,0,B1,brain,ACG,NaN,...,PATO:0000383 (female),NaN,MONDO:0005180,UBERON:0009835,NaN,EFO:0008913,nucleus,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3611,zupam_IPL,P78/11,zupam,rep2,2,1,B2,brain,IPL,NaN,...,PATO:0000383 (female),NaN,MONDO:0005180,UBERON:0006088,NaN,EFO:0008913,nucleus,NaN,NaN,NaN
3612,zupam_IPL,P78/11,zupam,rep2,2,1,B2,brain,IPL,NaN,...,PATO:0000383 (female),NaN,MONDO:0005180,UBERON:0006088,NaN,EFO:0008913,nucleus,NaN,NaN,NaN
3613,zupam_IPL,P78/11,zupam,rep2,2,1,B2,brain,IPL,NaN,...,PATO:0000383 (female),NaN,MONDO:0005180,UBERON:0006088,NaN,EFO:0008913,nucleus,NaN,NaN,NaN
3614,zupam_IPL,P78/11,zupam,rep2,2,1,B2,brain,IPL,NaN,...,PATO:0000383 (female),NaN,MONDO:0005180,UBERON:0006088,NaN,EFO:0008913,nucleus,NaN,NaN,NaN


In [152]:
# SAMPLE['age_at_diagnosis'].unique()
# # SAMPLE['age_at_diagnosis'] = SAMPLE['age_at_diagnosis'].replace({"Fastq":"fastq"})
SAMPLE.replace("NA", "", inplace=True)

# Testing the function with CLINPATH.csv and CDE.csv
validate_table(SAMPLE, "SAMPLE", CDE)

Missing Required Fields in SAMPLE: source_sample_id

Required Fields with Empty or NaN values:
replicate: 3256 rows
source_RIN: 3616 rows
header: 3616 rows
annotation: 3616 rows
configuration_file: 3616 rows
self_reported_ethnicity_ontology_term_id: 3616 rows
disease_ontology_term_id: 64 rows
cell_type_ontology_term_id: 3616 rows

Optional fields with Empty or NaN values:
pm_PH: 3616 rows
donor_id: 3616 rows

Invalid Field/Value pairs:
molecular_source: nuclear RNA
sequencing_end: paired-end
sequencing_length: 190
technology: sN
adjustment: raw
content: reads
organism_ontology_term_id: HUMAN NCBITaxon:9606


In [80]:
STUDY.head()

,name,value
0,project_name,Understanding mechanisms of Parkinson's diseas...
1,project_dataset,Hardy snRNA-seq
2,project_description,Genetic analysis has identified many risk gene...
3,ASAP_team_name,TEAM-HARDY
4,ASAP_lab_name,Ryten Lab


In [81]:
import hashlib
from pathlib import Path

def compute_md5(file_path):
    """
    Compute the MD5 hash of a file.
    
    Args:
    - file_path (str/Path): Path to the file.
    
    Returns:
    - str: MD5 hash of the file.
    """
    hash_md5 = hashlib.md5()
    with open(file_path, "rb") as f:
        for chunk in iter(lambda: f.read(4096), b""):
            hash_md5.update(chunk)
    return hash_md5.hexdigest()

# ALTERNATIVE call
# Example usage
base_path = Path('/path/to/files')
data['file_MD5'] = data['file_name'].apply(lambda x: compute_md5(base_path / x))


def compute_md5_mac(file_path):
    """
    Compute the MD5 hash of a file using the `md5` command on a Mac.
    
    Args:
    - file_path (str/Path): Path to the file.
    
    Returns:
    - str: MD5 hash of the file.
    """
    result = subprocess.run(['md5', '-q', file_path], capture_output=True, text=True)
    return result.stdout.strip()

# Example usage (to be run in your local environment):
# base_path = Path('/path/to/files')
# data['file_MD5'] = data['file_name'].apply(lambda x: compute_md5_mac(base_path / x))


NameError: name 'data' is not defined

In [ ]:

# fix STUDY formatting
tmp = pd.DataFrame()
tmp = STUDY[["name","value"]].transpose().reset_index().drop(columns=["index"])
tmp

In [ ]:

tmp.columns = tmp.iloc[0]
STUDY = tmp.drop([0])
STUDY.head()


In [ ]:
# Testing the function with STUDY.csv and CDE.csv
validate_table(STUDY, "STUDY", CDE)


# Need to add contributor_names

In [ ]:
PROTOCOL.head()

In [ ]:
# fix STUDY formatting
tmp = pd.DataFrame()
tmp = PROTOCOL[["name","value"]].transpose().reset_index().drop(columns=["index"])
tmp.columns = tmp.iloc[0]
PROTOCOL = tmp.drop([0])
PROTOCOL.head()

In [ ]:
# Testing the function with PROTOCOL.csv and CDE.csv
validate_table(PROTOCOL, "PROTOCOL", CDE)

In [ ]:
# Testing the function with SUBJECT.csv and CDE.csv
validate_table(SUBJECT, "SUBJECT", CDE)

In [ ]:
# Testing the function with SAMPLE.csv and CDE.csv
validate_table(SAMPLE, "SAMPLE", CDE)

# sequence length will need to be converted to a string

# add 'replicate' coding (nans)

In [ ]:
len(SAMPLE["file_name"].unique())

In [ ]:
# Testing the function with CLINPATH.csv and CDE.csv
validate_table(CLINPATH, "CLINPATH", CDE)

basically hold the list of the GP2ID and the original clinical ID pairs + how many samples are in the GP2 (s1 only or s1, s2,...) for all GP2 submitted individuals. It takes the sample manifest, scan the clinical ID to check if this is the additional submission of those already in the GP2 or not and then if its new, give new GP2ID and GP2sampleID. If the clinical_id is already existing in the GP2 then only provide GP2sampleID (GP2ID_sX+1). Also it errors if the original sample ID submitted is equal to the one in the list. (No duplication of sample ID from the same cohort)

In [4]:
team_names = ["team-lee", "team-hafler", "team-hardy", "team-jakobsson", "team-sherzer","team-sulzer", "tam-voet","team-wood"]

In [5]:
team_code = ["LEE", "HAF", "HAR", "JAK", "SHE", "SUL", "VOE", "WOO"]




In [7]:
[x.upper() for x in team_names]


['TEAM-LEE',
 'TEAM-HAFLER',
 'TEAM-HARDY',
 'TEAM-JAKOBSSON',
 'TEAM-SHERZER',
 'TEAM-SULZER',
 'TAM-VOET',
 'TEAM-WOOD']